In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

results = pd.DataFrame(index=['PacketsSent', 'PacketsSentFLC', 'PacketsReceived', 'PacketsReceivedFLC', 
                              'PacketsLost(%)', 'PacketsLostFLC(%)', 'Jitter', 'JitterFLC', 'Latency', 'LatencyFLC',
                              'PowerConsumed(W)', 'PowerConsumedFLC(W)'])

wsn2 = pd.read_csv('wsn2.csv', usecols=['run', 'type', 'module', 'name', 'value', 'mean'])

run = "wsnSc2T"

for i in range(1, 11):
    packetSentCount = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-1')) & (wsn2.type=='scalar') & 
                             (wsn2.module.str.startswith("Sc2.sensor")) & (wsn2.module.str.endswith("udp")) & 
                             (wsn2.name=='packetSent:count')]   

    packetReceivedCount = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-1'))& (wsn2.type=='scalar') & 
                                 (wsn2.module.str.startswith("Sc2.sink")) & 
                                 (wsn2.module.str.endswith("udp")) & (wsn2.name=='packetReceived:count')]

    totalPkSentCount = sum(packetSentCount.value)
    totalPkReceivedCount = sum(packetReceivedCount.value)
    packetLoss = (totalPkSentCount - totalPkReceivedCount) * 100 / totalPkSentCount

    residualEnergy = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-1')) & (wsn2.type=='scalar') & 
                            (wsn2.name=='residualEnergyCapacity:last') & 
                            (wsn2.module.str.endswith('energyStorage'))] 

    spentEnergyJ = 9000 - sum(residualEnergy.value)
    consumedPower = spentEnergyJ / 240

    latency = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-1')) & (wsn2.type=='histogram') & 
                     (wsn2.name=='endToEndDelay:histogram')] 

    latencyMean = latency["mean"].mean()
    
    jitter = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-1')) & (wsn2.type=='histogram') & 
                    (wsn2.name=='jitter:histogram')] 
    
    jitterMean = jitter["mean"].mean()

                        ########################################################
                        ########Simulation with FUZZY LOGIC CONTROLLER##########
                        ########################################################
    
    packetSentCountFLC = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-0')) & (wsn2.type=='scalar') & 
                                (wsn2.module.str.startswith("Sc2.sensor")) & 
                                (wsn2.module.str.endswith("udp")) & (wsn2.name=='packetSent:count')] 

    packetReceivedCountFLC = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-0'))& (wsn2.type=='scalar') & 
                                    (wsn2.module.str.startswith("Sc2.sink")) & 
                                    (wsn2.module.str.endswith("udp")) & (wsn2.name=='packetReceived:count')]

    totalPkSentCountFLC = sum(packetSentCountFLC.value)
    totalPkReceivedCountFLC = sum(packetReceivedCountFLC.value)
    packetLossFLC = (totalPkSentCountFLC - totalPkReceivedCountFLC) * 100 / totalPkSentCountFLC

    residualEnergyFLC = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-0')) & (wsn2.type=='scalar') & 
                               (wsn2.name=='residualEnergyCapacity:last') & 
                               (wsn2.module.str.endswith('energyStorage'))] 

    spentEnergyJFLC = 9000 - sum(residualEnergyFLC.value)
    consumedPowerFLC = spentEnergyJFLC / 240

    latencyFLC = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-0')) & (wsn2.type=='histogram') & 
                        (wsn2.name=='endToEndDelay:histogram')] 

    latencyMeanFLC = latencyFLC["mean"].mean()

    jitterFLC = wsn2[(wsn2.run.str.startswith('wsnSc2T'+str(i)+'-0')) & 
                       (wsn2.type=='histogram') & (wsn2.name=='jitter:histogram')] 

    jitterMeanFLC = jitterFLC["mean"].mean()   
    
                        ########################################################
                        ########                 RESULTS             ##########
                        ########################################################
            
    runDataCollector = [totalPkSentCount, totalPkSentCountFLC, totalPkReceivedCount, totalPkReceivedCountFLC,
                        packetLoss, packetLossFLC, jitterMean, jitterMeanFLC, latencyMean, latencyMeanFLC, 
                        consumedPower, consumedPowerFLC]
    results.insert(i-1, "T"+str(i), runDataCollector)
        
print(results)


                              T1           T2           T3           T4  \
PacketsSent          1294.000000  1683.000000  4349.000000  4854.000000   
PacketsSentFLC       1298.000000  1642.000000  4211.000000  5036.000000   
PacketsReceived      1239.000000  1571.000000  4017.000000  4498.000000   
PacketsReceivedFLC   1278.000000  1547.000000  3268.000000  4744.000000   
PacketsLost(%)          4.250386     6.654783     7.633939     7.334157   
PacketsLostFLC(%)       1.540832     5.785627    22.393731     5.798253   
Jitter                  0.017961     0.020247     0.032447     0.026590   
JitterFLC               0.021395     0.028510     0.022292     0.037447   
Latency                 0.042244     0.051274     0.108032     0.117086   
LatencyFLC              0.057158     0.085634     0.171168     0.172012   
PowerConsumed(W)        4.668130     4.728399     5.017736     5.078508   
PowerConsumedFLC(W)     2.624373     2.647080     2.785950     2.878227   

                        